# “analysis ready data” (ARD) workflow for ensemble ACCESS-ESM1.5 data

## GitHub Issues:
- https://github.com/shared-climate-data-problems/CMIP-data-problems/issues/8
- https://github.com/COSIMA/cosima-recipes/issues/444

Date: 19 September, 2024

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

### Goals:
- Write a `Jupyter` notebook example to show and explore best-practice
- importantly, maintain the original names for the 40 ensembles in the resulting `xarray` object, for example `r1i1p1f1`

#### Setting up your ARE session - be sure to dial up your `JobFS` resources

### Bookmark this information on NCI Queue limits
https://opus.nci.org.au/pages/viewpage.action?pageId=236881198

**See JobFS limits for your chosen queue**

# Setup your Dask cluster

NB: looks like a work-around in netcdf4-python to deal with netcdf-c not being thread safe was removed in 1.6.1. The solution (for now) is to make sure your cluster only uses 1 thread per worker

Discussion:
https://forum.access-hive.org.au/t/netcdf-not-a-valid-id-errors/389

![open source dependency XKCD](https://www.explainxkcd.com/wiki/images/d/d7/dependency.png)

Credit: XKCD - http://www.xkcd.com

In [1]:
### setup dask cluster
from dask.distributed import Client
client = Client(threads_per_worker=1) # for loading a dataset object from many NetCDF file paths
#client = Client() # possibly better for analysis after loading from an ARD collection / intermediate file
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34729,Workers: 28
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:46719,Total threads: 1
Dashboard: /proxy/43151/status,Memory: 8.97 GiB
Nanny: tcp://127.0.0.1:46379,


# utilise CMIP6 data catalogs for NCI holdings

The global CMIP6 (Coupled Model Intercomparison Project Phase 6) dataset is enormous, reflecting the extensive range of climate simulations and variables it contains. As of recent estimates, the CMIP6 archive is expected to surpass **20 petabytes (PB)** in total size. This includes data from various modeling centers around the world, covering different experiments, variables, and temporal resolutions.

The total number of NetCDF files in the entire CMIP6 dataset is rather large.

##### Information on climate data catalogs across Australian HPC

**ACCESS-NRI** https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/how.html <br>
**NCI** https://opus.nci.org.au/pages/viewpage.action?pageId=213713098


##### Earth System Grid Federation (ESGF) Australian CMIP6-era Datasets
https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f3154_9976_7262_7595
##### $\bigstar$ Get inspiration from ACCESS-NRI intake catalog docs: ACCESS-ESM1-5 CMIP6 example
https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/quickstart.html

### import some packages

In [3]:
import intake
import xarray as xr
import numpy as np
import gc

### import the ACCESS-NRI catalog

In [4]:
catalog = intake.cat.access_nri

### 

In [5]:
cmip6_datastore = catalog.search(name="cmip6.*", model="ACCESS-ESM1-5").to_source()

In [8]:
cmip6_datastore_filtered = cmip6_datastore.search(
    source_id="ACCESS-ESM1-5", 
    table_id="Omon", 
    variable_id="tos", 
    experiment_id="historical", 
    file_type="l"
)

cmip6_datastore_filtered

,unique
path,40
file_type,1
realm,1
frequency,1
table_id,1
project_id,1
institution_id,1
source_id,1
experiment_id,1
member_id,40


In [11]:
cmip6_datastore_filtered.keys()

['l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r10i1p1f1.mon.ocean.Omon.tos.gn.v20200605',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r11i1p1f1.mon.ocean.Omon.tos.gn.v20200803',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r12i1p1f1.mon.ocean.Omon.tos.gn.v20200803',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r13i1p1f1.mon.ocean.Omon.tos.gn.v20200803',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r14i1p1f1.mon.ocean.Omon.tos.gn.v20200803',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r15i1p1f1.mon.ocean.Omon.tos.gn.v20200803',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r16i1p1f1.mon.ocean.Omon.tos.gn.v20200803',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r17i1p1f1.mon.ocean.Omon.tos.gn.v20200803',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r18i1p1f1.mon.ocean.Omon.tos.gn.v20200803',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r19i1p1f1.mon.ocean.Omon.tos.gn.v20200803',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r1i1p1f1.mon.ocean.Omon.tos.gn.v20191115',
 'l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r20i1p1f1.mon.ocean.Omon.tos.gn.v2020

In [9]:
%%time
ds = xr.concat(
    cmip6_datastore_filtered.to_dataset_dict(progressbar=False).values(), 
    dim="member"
).rename({"tos": "sst"})

CPU times: user 10.6 s, sys: 1.87 s, total: 12.5 s
Wall time: 28.9 s


In [10]:
ds

<xarray.Dataset> Size: 34GB
Dimensions:             (member: 40, time: 1980, bnds: 2, j: 300, i: 360,
                         vertices: 4)
Coordinates:
  * time                (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 20...
  * j                   (j) int32 1kB 0 1 2 3 4 5 6 ... 294 295 296 297 298 299
  * i                   (i) int32 1kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
    latitude            (j, i) float64 864kB -77.88 -77.88 ... 65.63 65.21
    longitude           (j, i) float64 864kB 80.5 81.5 82.5 ... 79.97 79.99
Dimensions without coordinates: member, bnds, vertices
Data variables:
    time_bnds           (member, time, bnds) datetime64[ns] 1MB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    vertices_latitude   (member, j, i, vertices) float64 138MB dask.array<chunksize=(1, 300, 360, 2), meta=np.ndarray>
    vertices_longitude  (member, j, i, vertices) float64 138MB dask.array<chunksize=(1, 300, 360, 2), meta=np.ndarray>
    sst                 (member, time, j, i) float32 34GB dask.array<chunksize=(1, 1, 300, 360), meta=np.ndarray>
Attributes: (12/64)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            43829.0
    creation_date:                    2020-05-29T07:04:26Z
    ...                               ...
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         v20200529
    intake_esm_attrs:time_range:      185001-201412
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r...

In [12]:
# Convert the catalog to a dictionary
datasets_dict = cmip6_datastore_filtered.to_dataset_dict(progressbar=False)

# Extract the member names from the keys
member_names = [key.split('.')[5] for key in datasets_dict.keys()]

# Concatenate the datasets along the new 'member' dimension
ds = xr.concat(datasets_dict.values(), dim="member")

# Assign the member names to the 'member' coordinate
ds = ds.assign_coords(member=("member", member_names))

# Now, 'ds' will have the 'member' coordinate with the correct member names

In [13]:
ds

<xarray.Dataset> Size: 34GB
Dimensions:             (member: 40, time: 1980, bnds: 2, j: 300, i: 360,
                         vertices: 4)
Coordinates:
  * time                (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 20...
  * j                   (j) int32 1kB 0 1 2 3 4 5 6 ... 294 295 296 297 298 299
  * i                   (i) int32 1kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
    latitude            (j, i) float64 864kB -77.88 -77.88 ... 65.63 65.21
    longitude           (j, i) float64 864kB 80.5 81.5 82.5 ... 79.97 79.99
  * member              (member) <U9 1kB 'r39i1p1f1' 'r20i1p1f1' ... 'r7i1p1f1'
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (member, time, bnds) datetime64[ns] 1MB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    vertices_latitude   (member, j, i, vertices) float64 138MB dask.array<chunksize=(1, 300, 360, 2), meta=np.ndarray>
    vertices_longitude  (member, j, i, vertices) float64 138MB dask.array<chunksize=(1, 300, 360, 2), meta=np.ndarray>
    tos                 (member, time, j, i) float32 34GB dask.array<chunksize=(1, 1, 300, 360), meta=np.ndarray>
Attributes: (12/64)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            299499.0
    creation_date:                    2021-05-25T08:04:19Z
    ...                               ...
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         v20210525
    intake_esm_attrs:time_range:      185001-201412
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r...

In [ ]:
cmip6_fs38_catalog = intake.open_esm_datastore("/g/data/dk92/catalog/v2/esm/cmip6-fs38/catalog.json")

In [ ]:
cmip6_fs38_catalog

In [ ]:
cmip6_fs38_catalog.df

### What is the name for SST? Variable name search here: https://pcmdi.llnl.gov/mips/cmip3/variableList.html

In [ ]:
my_cmip6_search = cmip6_fs38_catalog.search(
experiment_id="historical",
realm='ocean',
source_id='ACCESS-CM2',
variable_id='tos',
frequency='mon',
file_type='f'
)
my_cmip6_search

In [ ]:
my_cmip6_search.unique().path

In [ ]:
%%time
dataset_dict = my_cmip6_search.to_dataset_dict(progressbar=False)
SST_ds = xr.concat(
    dataset_dict.values(),
    dim='member'
)

## ARD task - "Cleaning up" inconsistent files 

In [ ]:
%%time
dataset_dict = my_cmip6_search.to_dataset_dict(progressbar=False)
dataset_dict_clean = {key: ds.drop_vars(['vertices_longitude', 'vertices_latitude', 'time_bnds'], errors='ignore') 
           for key, ds in dataset_dict.items()}
SST_ds = xr.concat(dataset_dict_clean.values(), dim='member')

In [ ]:
SST_ds

## what is the default NetCDF chunking?
`ncdump -hs /g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r5i1p1f1/Omon/tos/gn/files/d20210607/tos_Omon_ACCESS-CM2_historical_r5i1p1f1_gn_185001-201412.nc`

**tos:_ChunkSizes = 1, 300, 360 ;**

In [ ]:
%%time
xarray_open_kwargs = {'chunks':{'member':1,'time':220,'j':300,'i':360}}
dataset_dict = my_cmip6_search.to_dataset_dict(progressbar=False,xarray_open_kwargs=xarray_open_kwargs)
dataset_dict_clean = {key: ds.drop_vars(['vertices_longitude', 'vertices_latitude', 'time_bnds'], errors='ignore') 
           for key, ds in dataset_dict.items()}
SST_ds_chunked_on_open = xr.concat(dataset_dict_clean.values(), dim='member')

In [ ]:
SST_ds_chunked_on_open

# Calculate monthly climatology - ARD makes it faster

In [ ]:
SST_climatology = SST_ds.groupby('time.month').mean('time')
SST_climatology

### Wall time: 3min 25s

In [ ]:
%%time
SST_climatology = SST_climatology.compute()

In [ ]:
gc.collect()

### Wall time: 32.3 s

In [ ]:
SST_ARD_climatology = SST_ds_chunked_on_open.groupby('time.month').mean('time')
SST_ARD_climatology

In [ ]:
%%time
SST_ARD_climatology = SST_ARD_climatology.compute()

# Explore SST climatology over the historical record for ACCESS-CM2

In [ ]:
(SST_ARD_climatology.tos.isel({'month':0,'member':0}) - SST_ARD_climatology.tos.isel({'month':0,'member':5})).plot()

# Ensemble calculations - "what's the spread in SST" - ARD makes it POSSIBLE

# Rechunking for ARD ensemble calculations - XL Cluster (14 cpu / 63 GB) used - Wall time: 1min 2s

In [ ]:
def remove_encoding(DS):
    for var in DS:
        DS[var].encoding = {}

    for coord in DS.coords:
        DS[coord].encoding = {}
    return DS

In [ ]:
SST_ds_rechunked = SST_ds_chunked_on_open.chunk({'member':10,'time':220,'j':300,'i':36})
SST_ds_rechunked

In [ ]:
SST_ds_rechunked.tos.encoding

In [ ]:
remove_encoding(SST_ds_rechunked)
SST_ds_rechunked.tos.encoding

In [ ]:
%%time
SST_ds_rechunked.to_zarr('/scratch/nf33/moore_tutorial/SST_ARD_rechunked.zarr',consolidated=True)

# open ARD Zarr collection for SST

In [ ]:
SST_ds_rechunked = xr.open_zarr('/scratch/nf33/moore_tutorial/SST_ARD_rechunked.zarr',consolidated=True)

In [ ]:
SST_ds_rechunked

In [ ]:
SST_ds_chunked_on_open

In [ ]:
SST_spread = SST_ds_chunked_on_open.std(dim='member')
SST_spread_rechunked = SST_ds_rechunked.std(dim='member')

In [ ]:
%%time
SST_spread.tos.mean('time').plot()

In [ ]:
%%time
SST_spread_rechunked.tos.mean('time').plot(robust=True)

# $The$ $End$

In [ ]:
client.shutdown()